In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv("../input/spaceship-titanic/train.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
df['Name'].value_counts()

In [ ]:
df.loc[df['Name'] == "Gollux Reedall"]

The name looks not important, we cannot deduce any information from it neither, it is more likely to produce noise, we will remove it for now

In [ ]:
df.drop(columns = ["Name"], axis = 1, inplace = True)
df.head()

PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.


The group and number could be meaningull as the transportations could be done in a specific order, we will extract them

In [ ]:
def split_column(column):
    group, number = [], []
    for cell in column:
        [tmp_group, tmp_number] = cell.split("_")
        group.append(int(tmp_group))
        number.append(int(tmp_number))
    return [group, number]
[group, number] = split_column(df["PassengerId"])
df["group"] = group
df["number"] = number
df.drop(columns = ["PassengerId"], axis = 1, inplace = True)
df.head()

label encode true/false

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Transported"] = le.fit_transform(df["Transported"])
df.head()

In [ ]:
df["Transported"].value_counts()

In [ ]:
df["VRDeck"].value_counts()

In [ ]:
df["Spa"].value_counts()

In [ ]:
df["ShoppingMall"].value_counts()

In [ ]:
df["RoomService"].value_counts()

In [ ]:
df["FoodCourt"].value_counts()

these upper columns could define importance of clients, it should be kept along with vip

In [ ]:
df["VIP"].value_counts()

In [ ]:
set(df["VIP"])

we can proceed by knn imputer but for this simple approach we'll pick the dominant value which is false

In [ ]:
df["VIP"].fillna(False, inplace = True)

In [ ]:
set(df["VIP"])

In [ ]:
df["VIP"].value_counts()

In [ ]:
le2 = LabelEncoder()
df["VIP"] = le2.fit_transform(df["VIP"])
df.head()


we noticed abberant values in the age from .describe, let's check it out

In [ ]:
df.boxplot(column=['Age'], return_type='axes');

In [ ]:
print(sorted(df["Age"])[:10])

In [ ]:
print(sorted(df["Age"], reverse = True)[:10])

Age of 0????????????

In [ ]:
df.loc[df["Age"] == 0]

In [ ]:
df.loc[df["Age"] == 0]["Transported"].value_counts()

could be babies privilege... keep it as it is but as there are alot of abberant values, we will be using the mediant to fill ( again knn impute could be used)

In [ ]:
df["Age"].value_counts()

In [ ]:
df["Age"].median()

In [ ]:
df["Age"].fillna(df["Age"].median(), inplace = True)
df["Age"].isna().sum()

In [ ]:
df["Age"].value_counts()

In [ ]:
df.head()

In [ ]:
df["Destination"].value_counts()

encodable

In [ ]:
df["Destination"].isna().sum()

dom value strat, tendencies??

In [ ]:
df["Destination"].fillna("TRAPPIST-1e", inplace = True)

In [ ]:
df["Destination"].value_counts()

In [ ]:
df["Destination"].isna().sum()

In [ ]:
le3 = LabelEncoder()
df["Destination"] = le3.fit_transform(df["Destination"])
df["Destination"].value_counts()

In [ ]:
df.head()

Cabin = deck/num/side, separate them and check counts

In [ ]:
df["Cabin"].isna().sum()

In [ ]:
df["Cabin"].value_counts()

information could be usefull, however due to the sparsity of the values regarding the total, this is more prone to produce noise unless we could regroup them, for simplicity, we will drop this. However a closer inspection should be done for improvements.

In [ ]:
df.drop(columns = ["Cabin"], axis = 1, inplace = True)
df.head()

In [ ]:
df["CryoSleep"].isna().sum()

In [ ]:
df["CryoSleep"].value_counts()

In [ ]:
df["CryoSleep"].fillna(False, inplace = True)#dom value
le4 = LabelEncoder()
df["CryoSleep"] = le4.fit_transform(df["CryoSleep"])
df["CryoSleep"].value_counts()

In [ ]:
df["HomePlanet"].isna().sum()

In [ ]:
df["HomePlanet"].value_counts()

In [ ]:
df["HomePlanet"].fillna("Earth", inplace = True)
le5 = LabelEncoder()
df["HomePlanet"] = le5.fit_transform(df["HomePlanet"])
df["HomePlanet"].value_counts()

In [ ]:
df.isna().sum()

the columns that we didn't touch still has missing values since they are float, we will simply fill with mean even though they are skewed

In [ ]:
df["RoomService"].fillna(df["RoomService"].mean(), inplace = True)
df["FoodCourt"].fillna(df["FoodCourt"].mean(), inplace = True)
df["ShoppingMall"].fillna(df["ShoppingMall"].mean(), inplace = True)
df["Spa"].fillna(df["Spa"].mean(), inplace = True)
df["VRDeck"].fillna(df["VRDeck"].mean(), inplace = True)
df.isna()

In [ ]:
df.isna().sum().mean()

done doing the simplest preprocessing possible

In [ ]:
target = df["Transported"]

In [ ]:
df.drop(columns = ["Transported"], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = scaler.fit_transform(df)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, random_state = 0)
print(X_train.shape, set(y_train))

In [ ]:
print(X_test.shape, set(y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression as lr
clf = lr()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report as cr
print(cr(preds, y_test))

In [ ]:
print(cr(clf.predict(X_train), y_train))

In [ ]:
clf = lr()
clf.fit(df, target)

we will predict after processing the test csv

In [ ]:
test_df = pd.read_csv("../input/spaceship-titanic/test.csv")
test_df.head()

In [ ]:
test_df.isna().sum()

In [ ]:
test_df.drop(columns = ["Name"], axis = 1, inplace = True)
test_df.head()

In [ ]:
[group, number] = split_column(test_df["PassengerId"])
test_df["group"] = group
test_df["number"] = number
test_df.drop(columns = ["PassengerId"], axis = 1, inplace = True)
test_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
test_df["VIP"].fillna(False, inplace = True)

In [ ]:
le2 = LabelEncoder()
test_df["VIP"] = le2.fit_transform(test_df["VIP"])
test_df.head()


In [ ]:
test_df["Age"].fillna(test_df["Age"].median(), inplace = True)
test_df["Age"].isna().sum()

In [ ]:
test_df["Destination"].fillna("TRAPPIST-1e", inplace = True)

In [ ]:
le3 = LabelEncoder()
test_df["Destination"] = le3.fit_transform(test_df["Destination"])
test_df["Destination"].value_counts()

In [ ]:
test_df.drop(columns = ["Cabin"], axis = 1, inplace = True)
test_df.head()

In [ ]:
test_df["CryoSleep"].fillna(False, inplace = True)#dom value
le4 = LabelEncoder()
test_df["CryoSleep"] = le4.fit_transform(test_df["CryoSleep"])
test_df["CryoSleep"].value_counts()

In [ ]:
test_df["HomePlanet"].fillna("Earth", inplace = True)
le5 = LabelEncoder()
test_df["HomePlanet"] = le5.fit_transform(test_df["HomePlanet"])
test_df["HomePlanet"].value_counts()

In [ ]:
test_df["RoomService"].fillna(test_df["RoomService"].mean(), inplace = True)
test_df["FoodCourt"].fillna(test_df["FoodCourt"].mean(), inplace = True)
test_df["ShoppingMall"].fillna(test_df["ShoppingMall"].mean(), inplace = True)
test_df["Spa"].fillna(test_df["Spa"].mean(), inplace = True)
test_df["VRDeck"].fillna(test_df["VRDeck"].mean(), inplace = True)
test_df.isna()

In [ ]:
test_df.isna().sum().mean()

In [ ]:
test_df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
test_df = scaler.fit_transform(test_df)

In [ ]:
preds = clf.predict(test_df)

In [ ]:
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
preds_converted  = []
for pred in preds:
    if pred == 1:
        preds_converted.append(True)
    else:
        preds_converted.append(False)
submission["Transported"] = preds_converted
submission.head()

In [ ]:
submission.to_csv("submissionSpace1.csv", index = False)